# Introduction

This is the final Capstone Project for the Coursera IBM Data Science course. To finish the course, I decided to use what I have learned on two topics very important to me: Food and my home of California. I decided to do my research on the following cities: Davis (my hometown), Santa Barbara (where I went to university), and San Francisco (my favorite city to visit). The basis of this project will be to get a better idea of potential suggestions for investors and entrepreneurs looking to make a successful investment in a restaurant. Overall, I will serach to see what type of restaurant would be the best and in which city to pwould make the most sense in searching for potential business property.

The project will attempt to answer the question of how accurately can someone predcit the amount of 'likes' a restaurant opening in these cities can expect to have based on the type of food/cuisine it will serve and which of the 3 Californian cities it will open in. Based on what I learned throughout the course, I will analyze and model the data using different forms of machine learning by comparing both linear and logistic regressions to see which method will yield more accurate preficitive capabilities after training and testing.

# Data

After doing this analysis, I will name this data frame 'raw_dataset' becuase it is the most indepth and complete form of data.

My first step will be to retrieve the geogrphical coordinates of the 3 mentioned cities (Davis, Santa Barbara, and San Francisco). I will then go ahead and leverage the Foursquare API to obtain the URLs that give me the raw data in JSON form. These URLs will then need to be scraped for the columns: 'name', 'categories', 'latitude', 'longitude', and 'id' for each of the 3 cities. These columns will help identify where each restuarnt is from when doing research on different restaurants from each city.

I will limit my randge of all restaurants found within a 500km radius from the coordinates that are provided from the geolocator. Foursquare provides many categories that are not useful for the data I am looking for, so I will have to clean the results by removing all the rows that are not restaurants. From there, I will need the 'likes' data to come to a final conclusion on what type of food and where the restaurant should be located for our final decision of the business problem.

The 'id' column will be used to pull the 'likes' in the Foursquare API and append the information into the data frame. I will then conclude by naming the dataframe 'raw_dataset', just I did in the machine learning part of my project.

# Methodology

I will be using both linear and logistic regression to train and test the data. Linear regression was used to predict the number of 'likes' a new restaurant in this region will recieve, and Sci-Kit Learn will be employed to help with the stage. 

Logistic regression is also used as the classification method. Due to the use of binning when classifying by number of 'likes', I will essentially make use of multinomial logistic regression to perform my analysis. Although the ranges are discrete categories, they can be considered ordinal in nature. The logistic regression will need to be specified as being both multinomial and ordinal. The Sci-Kit Learn package is also perfect for this.

In [1]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.1 MB/s  eta 0:00:01


In [2]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
import matplotlib.pyplot as plt
import pylab as pl
import itertools
import warnings


from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score

print('All libraries have been imported')

All libraries have been imported


### Foursquare City Coordinates 

In [3]:
address1 = 'Davis, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))

address2 = 'Santa Barbara, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'San Francisco, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))

The geograpical coordinate of Davis, California are 38.545379, -121.7445835.
The geograpical coordinate of Santa Barbara, California are 34.4221319, -119.7026673.
The geograpical coordinate of San Francisco, California are 37.7790262, -122.419906.


### Foursquare Credentials 

In [4]:
CLIENT_ID = 'P4NGPZ24ZB0TWOPRDHZRBOAPXNTE0LTMZNSS1NUDON01CSUS' # your Foursquare ID
CLIENT_SECRET = 'GAQPVHHHR4M1RR1TPDBJUBOXUHSHOWO1Q0PWBKMGLIFSKAIG' # your Foursquare Secret
VERSION = '20120609' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)


# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

print(url1, url2, url3)

Your credentails:
CLIENT_ID: P4NGPZ24ZB0TWOPRDHZRBOAPXNTE0LTMZNSS1NUDON01CSUS
CLIENT_SECRET:GAQPVHHHR4M1RR1TPDBJUBOXUHSHOWO1Q0PWBKMGLIFSKAIG
https://api.foursquare.com/v2/venues/explore?&client_id=P4NGPZ24ZB0TWOPRDHZRBOAPXNTE0LTMZNSS1NUDON01CSUS&client_secret=GAQPVHHHR4M1RR1TPDBJUBOXUHSHOWO1Q0PWBKMGLIFSKAIG&v=20120609&ll=38.545379,-121.7445835&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=P4NGPZ24ZB0TWOPRDHZRBOAPXNTE0LTMZNSS1NUDON01CSUS&client_secret=GAQPVHHHR4M1RR1TPDBJUBOXUHSHOWO1Q0PWBKMGLIFSKAIG&v=20120609&ll=34.4221319,-119.7026673&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=P4NGPZ24ZB0TWOPRDHZRBOAPXNTE0LTMZNSS1NUDON01CSUS&client_secret=GAQPVHHHR4M1RR1TPDBJUBOXUHSHOWO1Q0PWBKMGLIFSKAIG&v=20120609&ll=37.7790262,-122.419906&radius=1000&limit=100


### Data Exploration 

In [5]:
# scrape the data from the generated URLs

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# FIRST CITY   

venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = pd.json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 = nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

# SECOND CITY

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = pd.json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 = nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

# THIRD CITY

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = pd.json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 = nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]





print('{} venues for Davis, California were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print()
print('{} venues for Santa Barbara, California were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print()
print('{} venues for San Francisco, California were returned by Foursquare.'.format(nearby_venues3.shape[0]))

80 venues for Davis, California were returned by Foursquare.

100 venues for Santa Barbara, California were returned by Foursquare.

100 venues for San Francisco, California were returned by Foursquare.


In [6]:
# add locations data to the data sets of each city

nearby_venues1['city'] = 'Davis'
nearby_venues2['city'] = 'Santa Barbara'
nearby_venues3['city'] = 'San Francisco'

In [7]:
# combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)

In [8]:
nearby_venues

,name,categories,lat,lng,id,city
0,Davis Farmers' Market,Farmers Market,38.544677,-121.744177,4b0586bcf964a520e96b22e3,Davis
1,Burgers and Brew,Burger Joint,38.544259,-121.743501,4a5a43c4f964a52005ba1fe3,Davis
2,Crepeville,Creperie,38.544121,-121.743720,4b581a10f964a520f14a28e3,Davis
3,Yolo Berry Yogurt,Frozen Yogurt Shop,38.544726,-121.743731,4b649d61f964a52059c22ae3,Davis
4,Zia's Delicatessen,Sandwich Place,38.544581,-121.740720,4b1ad416f964a520b5f223e3,Davis
5,Blaze Pizza,Pizza Place,38.543927,-121.739961,551ed2fc498e41a1316bff74,Davis
6,Central Park,Park,38.545402,-121.744500,4b6ee64ff964a520d5cf2ce3,Davis
7,Sam's Cuisine,Middle Eastern Restaurant,38.544064,-121.745119,4db9cdee0cb65435c51e5b27,Davis
8,de Vere's Irish Pub,Irish Pub,38.543889,-121.741388,4d9b743bd888ba7a7c9ee13d,Davis
9,Philz Coffee,Coffee Shop,38.543392,-121.741341,5a4a89b4610f046421440639,Davis


In [9]:
nearby_venues['categories'].unique()

array(['Farmers Market', 'Burger Joint', 'Creperie', 'Frozen Yogurt Shop',
       'Sandwich Place', 'Pizza Place', 'Park',
       'Middle Eastern Restaurant', 'Irish Pub', 'Coffee Shop',
       'Ice Cream Shop', 'Mexican Restaurant', 'Sushi Restaurant',
       'Burrito Place', 'American Restaurant', 'Grocery Store',
       'Beer Store', 'Brewery', 'Café', 'Beer Bar', 'Thai Restaurant',
       'Chinese Restaurant', 'Bar', 'Garden', 'Bakery',
       'Andhra Restaurant', 'Sporting Goods Shop', 'Fast Food Restaurant',
       'Restaurant', 'Mediterranean Restaurant', 'Hawaiian Restaurant',
       'Indian Restaurant', 'Hot Dog Joint', 'Japanese Restaurant',
       'Health & Beauty Service', 'Spa', 'Indie Movie Theater',
       'Bubble Tea Shop', 'Salad Place', 'Italian Restaurant',
       'Thrift / Vintage Store', 'Student Center', 'Climbing Gym',
       'Shipping Store', 'Cupcake Shop', 'Hotel', 'Bike Shop',
       'Taiwanese Restaurant', 'Home Service', 'Courthouse',
       'Asian Restaura

### Data Cleaning 

In [10]:
# check list and manually remove all non-restaurant data

nearby_venues['categories'].unique()

removal_list = ['Clothing Store','Bar','Brewery', 
                'Comic Shop', 'Yoga Studio','Café', 'Accessories Store',
                'Coffee Shop', 'Tiki Bar', 'Music Venue', 'Student Center',
                'Wine Bar',  'Cocktail Bar', 'Dance Studio', 'Shipping Store',
                'Gym / Fitness Center','Beer Bar', 'Courthouse', 'Historic Site',
                'Bubble Tea Shop', 'Nightclub', 'Food Court', 'Bike Shop', 'Market',
                'Ice Cream Shop', 'Cupcake Shop', 'Skating Rink', 'Post Office',
                'Dessert Shop', 'Climbing Gym', 'Bakery', 'Home Service', 'Cheese Shop',
                'Farmers Market', 'Gay Bar','Beer Garden', 'Marijuana Dispensary',
                'Tea Room','Arts & Crafts Store', 'Grocery Store', 'Art Museum',
                'Sports Bar', 'Museum', 'Street Food Gathering', 'Event Space',
                'Library', 'Skate Park', 'Movie Theater','Park', 'Optical Shop',
                'Gym', 'Stadium', 'Furniture / Home Store', 'Discount Store', 
                'Playground', 'Cosmetics Shop', 'Casino', 'Souvlaki Shop', 'Fish Market',
                'Pet Store','Electronics Store', 'Snack Place', 'Food & Drink Shop',
                'Salon / Barbershop', 'Shopping Plaza', 'Deli / Bodega', 'Bank',
                'Candy Store', 'Liquor Store', 'Hotel', 'Music School', 'Rock Club', 
                'Shoe Store', 'Bookstore', 'Shopping Mall', 'Wine Shop', 'Butcher', 
                'Dive Bar', 'Video Game Store', 'Pharmacy', "Men's Store",  'Spiritual Center',
                'Accessories Store', 'Lingerie Store', 'Mobile Phone Shop', 'Bed & Breakfast',
                'Pool Hall', 'Juice Bar', 'Kids Store', 'Jazz Club', 'Boutique', 
                'Supplement Shop', 'Big Box Store', 'Mattress Store', 'Performing Arts Venue',
                'Hardware Store', 'Paper / Office Supplies Store', 'Theater', 'Luggage Store',
                'Business Service', 'Donut Shop', 'Beer Store', 'Opera House',
                'Lounge', 'Health Food Store', 'Pedestrian Plaza', 'Hotel Bar',
                'Hookah Bar', 'Concert Hall', 'Chocolate Shop', 'Sporting Goods Shop',
                'Hostel', 'Convenience Store', 'Pub', 'Garden', 'Health & Beauty Service',
                'Plaza', 'Comedy Club', 'Speakeasy', 'Spa', 'Indie Movie Theater', 
                'Tattoo Parlor', 'Massage Studio', 'Thrift / Vintage Store',]



nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique().tolist()

['Burger Joint',
 'Creperie',
 'Frozen Yogurt Shop',
 'Sandwich Place',
 'Pizza Place',
 'Middle Eastern Restaurant',
 'Irish Pub',
 'Mexican Restaurant',
 'Sushi Restaurant',
 'Burrito Place',
 'American Restaurant',
 'Thai Restaurant',
 'Chinese Restaurant',
 'Andhra Restaurant',
 'Fast Food Restaurant',
 'Restaurant',
 'Mediterranean Restaurant',
 'Hawaiian Restaurant',
 'Indian Restaurant',
 'Hot Dog Joint',
 'Japanese Restaurant',
 'Salad Place',
 'Italian Restaurant',
 'Taiwanese Restaurant',
 'Asian Restaurant',
 'Tapas Restaurant',
 'New American Restaurant',
 'Taco Place',
 'Noodle House',
 'French Restaurant',
 'Cajun / Creole Restaurant',
 'Diner',
 'Seafood Restaurant',
 'Mac & Cheese Joint',
 'Poke Place',
 'Ramen Restaurant',
 'Vietnamese Restaurant',
 'Dumpling Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Southern / Soul Food Restaurant',
 'Bagel Shop',
 'Fried Chicken Joint']

### Dataframe Creation 

In [11]:
# This next step is made to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[87, 77, 28, 24, 42, 42, 85, 11, 91, 11, 62, 45, 45, 14, 57, 10, 31, 57, 223, 8, 15, 10, 9, 25, 9, 34, 16, 40, 14, 45, 14, 7, 29, 18, 16, 0, 3, 0, 103, 67, 60, 119, 77, 97, 126, 53, 29, 200, 54, 30, 34, 24, 15, 70, 16, 30, 24, 36, 168, 57, 106, 74, 23, 63, 4, 43, 7, 36, 9, 56, 16, 5, 7, 4, 11, 27, 21, 7, 6, 12, 160, 143, 5, 67, 370, 482, 5, 674, 8, 110, 153, 1193, 371, 60, 72, 442, 4, 119, 117, 317, 18, 616, 8, 208, 95, 77, 20, 217, 117, 165]


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,name,categories,lat,lng,id,city,likes
1,Burgers and Brew,Burger Joint,38.544259,-121.743501,4a5a43c4f964a52005ba1fe3,Davis,87
2,Crepeville,Creperie,38.544121,-121.743720,4b581a10f964a520f14a28e3,Davis,77
3,Yolo Berry Yogurt,Frozen Yogurt Shop,38.544726,-121.743731,4b649d61f964a52059c22ae3,Davis,28
4,Zia's Delicatessen,Sandwich Place,38.544581,-121.740720,4b1ad416f964a520b5f223e3,Davis,24
5,Blaze Pizza,Pizza Place,38.543927,-121.739961,551ed2fc498e41a1316bff74,Davis,42


In [12]:
nearby_venues.count()

name          110
categories    110
lat           110
lng           110
id            110
city          110
likes         110
dtype: int64

In [13]:
# This next step will rename this raw data set into something more appropriate

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
1,Burgers and Brew,Burger Joint,38.544259,-121.743501,4a5a43c4f964a52005ba1fe3,Davis,87
2,Crepeville,Creperie,38.544121,-121.743720,4b581a10f964a520f14a28e3,Davis,77
3,Yolo Berry Yogurt,Frozen Yogurt Shop,38.544726,-121.743731,4b649d61f964a52059c22ae3,Davis,28
4,Zia's Delicatessen,Sandwich Place,38.544581,-121.740720,4b1ad416f964a520b5f223e3,Davis,24
5,Blaze Pizza,Pizza Place,38.543927,-121.739961,551ed2fc498e41a1316bff74,Davis,42


### Data Preparation for Machine learning 

In [14]:
# inspecting the raw dataset to see the amount of different types of cuisines

raw_dataset['categories'].unique().tolist()

['Burger Joint',
 'Creperie',
 'Frozen Yogurt Shop',
 'Sandwich Place',
 'Pizza Place',
 'Middle Eastern Restaurant',
 'Irish Pub',
 'Mexican Restaurant',
 'Sushi Restaurant',
 'Burrito Place',
 'American Restaurant',
 'Thai Restaurant',
 'Chinese Restaurant',
 'Andhra Restaurant',
 'Fast Food Restaurant',
 'Restaurant',
 'Mediterranean Restaurant',
 'Hawaiian Restaurant',
 'Indian Restaurant',
 'Hot Dog Joint',
 'Japanese Restaurant',
 'Salad Place',
 'Italian Restaurant',
 'Taiwanese Restaurant',
 'Asian Restaurant',
 'Tapas Restaurant',
 'New American Restaurant',
 'Taco Place',
 'Noodle House',
 'French Restaurant',
 'Cajun / Creole Restaurant',
 'Diner',
 'Seafood Restaurant',
 'Mac & Cheese Joint',
 'Poke Place',
 'Ramen Restaurant',
 'Vietnamese Restaurant',
 'Dumpling Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Southern / Soul Food Restaurant',
 'Bagel Shop',
 'Fried Chicken Joint']

In [15]:
# we can group some cuisines together to make a better categorical variable

european = ['Mediterranean Restaurant', 'Pizza Place', 'Creperie', 'Tapas Restaurant',
       'French Restaurant', 'Italian Restaurant', 'Irish Pub']

latin = ['Mexican Restaurant', 'Taco Place', 'Burrito Place']

asian = ['Japanese Restaurant', 'Vietnamese Restaurant', 'Chinese Restaurant', 'Taiwanese Restaurant', 'Poke Place',
         'Indian Restaurant', 'Middle Eastern Restaurant','Thai Restaurant', 'Dumpling Restaurant', 'Dim Sum Restaurant', 
         'Andhra Restaurant', 'Asian Restaurant', 'Filipino Restaurant', 'Sushi Restaurant','Noodle House', 'Ramen Restaurant']

american = ['Vegetarian / Vegan Restaurant', 'Seafood Restaurant', 'Caribbean Restaurant', 'Cajun / Creole Restaurant',
           'Burger Joint', 'American Restaurant', 'New American Restaurant', 'Hawaiian Restaurant', 'Mac & Cheese Joint',
            'Southern / Soul Food Restaurant', 'Diner', 'Salad Place']

casual = ['Bagel Shop', 'Sandwich Place', 'Fried Chicken Joint', 'Frozen Yogurt Shop',
          'Breakfast Spot', 'Wings Joint', 'Fast Food Restaurant', 'Restaurant']





def conditions(s):
    if s['categories'] in european:
        return 'european'
    if s['categories'] in latin:
        return 'latin'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in casual:
        return 'casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,name,categories,lat,lng,id,city,likes,categories_classified
1,Burgers and Brew,Burger Joint,38.544259,-121.743501,4a5a43c4f964a52005ba1fe3,Davis,87,american
2,Crepeville,Creperie,38.544121,-121.743720,4b581a10f964a520f14a28e3,Davis,77,european
3,Yolo Berry Yogurt,Frozen Yogurt Shop,38.544726,-121.743731,4b649d61f964a52059c22ae3,Davis,28,casual
4,Zia's Delicatessen,Sandwich Place,38.544581,-121.740720,4b1ad416f964a520b5f223e3,Davis,24,casual
5,Blaze Pizza,Pizza Place,38.543927,-121.739961,551ed2fc498e41a1316bff74,Davis,42,european
7,Sam's Cuisine,Middle Eastern Restaurant,38.544064,-121.745119,4db9cdee0cb65435c51e5b27,Davis,42,asian
8,de Vere's Irish Pub,Irish Pub,38.543889,-121.741388,4d9b743bd888ba7a7c9ee13d,Davis,85,european
11,Ike's Sandwiches,Sandwich Place,38.543902,-121.739976,57787262498e3c4f7717baec,Davis,11,casual
13,Woodstock's Pizza,Pizza Place,38.544218,-121.739211,4abfb6acf964a520c49120e3,Davis,91,european
14,Taqueria La Piñata,Mexican Restaurant,38.541693,-121.743830,551ed0bd498ee22d17a10fe3,Davis,11,latin


In [16]:
# Confirm the categories_classified has been created correctly

pd.crosstab(index = raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,27
asian,29
casual,16
european,25
latin,12


In [17]:
raw_dataset['likes'].mean()

89.02727272727273

In [18]:
# create a function to bin for us

def rankings(df):
    
    if df['likes'] <= 60:
        return 3
    
    elif df['likes'] <= 100:
        return 2
    
    elif df['likes'] > 100:
        return 1

In [19]:
# apply rankings function to dataset

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,name,categories,lat,lng,id,city,likes,categories_classified,ranking
1,Burgers and Brew,Burger Joint,38.544259,-121.743501,4a5a43c4f964a52005ba1fe3,Davis,87,american,2
2,Crepeville,Creperie,38.544121,-121.743720,4b581a10f964a520f14a28e3,Davis,77,european,2
3,Yolo Berry Yogurt,Frozen Yogurt Shop,38.544726,-121.743731,4b649d61f964a52059c22ae3,Davis,28,casual,3
4,Zia's Delicatessen,Sandwich Place,38.544581,-121.740720,4b1ad416f964a520b5f223e3,Davis,24,casual,3
5,Blaze Pizza,Pizza Place,38.543927,-121.739961,551ed2fc498e41a1316bff74,Davis,42,european,3
7,Sam's Cuisine,Middle Eastern Restaurant,38.544064,-121.745119,4db9cdee0cb65435c51e5b27,Davis,42,asian,3
8,de Vere's Irish Pub,Irish Pub,38.543889,-121.741388,4d9b743bd888ba7a7c9ee13d,Davis,85,european,2
11,Ike's Sandwiches,Sandwich Place,38.543902,-121.739976,57787262498e3c4f7717baec,Davis,11,casual,3
13,Woodstock's Pizza,Pizza Place,38.544218,-121.739211,4abfb6acf964a520c49120e3,Davis,91,european,2
14,Taqueria La Piñata,Mexican Restaurant,38.541693,-121.743830,551ed0bd498ee22d17a10fe3,Davis,11,latin,3


### Machine Learning: Linear Regression

In [20]:
# This will create a 'dummy' for linear regression modelling

# One Hot Encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# Next part will add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# This will move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset.head()

,name,american,asian,casual,european,latin,Davis,San Francisco,Santa Barbara,ranking,likes
1,Burgers and Brew,1,0,0,0,0,1,0,0,2,87
2,Crepeville,0,0,0,1,0,1,0,0,2,77
3,Yolo Berry Yogurt,0,0,1,0,0,1,0,0,3,28
4,Zia's Delicatessen,0,0,1,0,0,1,0,0,3,24
5,Blaze Pizza,0,0,0,1,0,1,0,0,3,42


In [21]:
# Multiple Linear Regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Davis', 
                         'Santa Barbara', 'San Francisco']])

y = np.asanyarray(train[['likes']])
regr.fit (x, y)

# The coefficients

print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 76.89597129  12.09406862   1.5684824   38.35850294   0.96252901
  -57.6411757  -52.10697645 109.74815215]]


In [22]:
# Multiple Linear Regression Prediction Capabilities

y_hat= regr.predict(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Davis', 
                         'Santa Barbara', 'San Francisco']])

x = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Davis', 
                         'Santa Barbara', 'San Francisco']])

y = np.asanyarray(test[['likes']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 18427.01
Variance score: 0.34


### Machine Learning: Logistic Regression 

In [23]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Davis', 
                         'Santa Barbara', 'San Francisco']])

y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Davis', 
                         'Santa Barbara', 'San Francisco']])

y_test = np.asanyarray(test['ranking'])


mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[ 0.09211704 -0.04257673  0.43870003 -0.14682631 -0.29359597 -0.8431859
 -0.11112851  0.95430115]


In [24]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob


# average = None, average = 'micro', average = 'macro', or average = 'weighted'
jaccard_score(y_test, yhat, average='weighted')

0.5638095238095236

In [25]:
log_loss(y_test, yhat_prob)

0.7757823653026273

In [26]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['american', 'asian', 'casual',
                                   'european', 'latin', 'Davis', 
                                   'Santa Barbara', 'San Francisco']])

y_all = np.asanyarray(reg_dataset['ranking'])



LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print(coef)

[ 0.24852352 -0.2102931   0.38429147 -0.0709045  -0.30649903 -0.86531325
 -0.16193357  1.02724688]


In [27]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.67      0.80      0.73         5
           2       0.00      0.00      0.00         2
           3       0.78      0.88      0.82         8

    accuracy                           0.73        15
   macro avg       0.48      0.56      0.52        15
weighted avg       0.64      0.73      0.68        15



/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Results

A linear regression model was trained on a random subsample of 80% and then the other 20% was used for testing purposes. In order to evaluate if the model is reasonable, the residual sum of squares and variance score were both calculated (18427.01, 0.34). The variance score is low, which means that it is not the best way of modeling our data. Therefore, I moved on to logistic regression for the analysis.

The multinomial ordinal logistic regression model was also trained on a random subsample of 80% and then tested on the remaining 20%. The jaccard score and log-loss were both calculated (56.38% and 0.775 respectively). Although the prediction is not promosing, a jaccard score of 56% is somewhat reasonable. The classification report is included in the analysis.

Given the modestly accurate ability of this mode, we have the ability to run the model on the complete dataset. The coefficients we got show that opening a restaurant in Davis or serving cuisine that is latin are the two factors negatively associated the most with 'likes'.



# Discussion

The first thing to note is that given the data, logistic regression presents a better fit for the data over linear regression. Using logistic regression we were able to obtain a Jaccard Score of 58.38%, which although not the best rating, is more reasonable than the low variance score obtained from the linear regression. As was mentioned before, it should be noted that for the purposes of this project, we are assumming that 'likes' are a good representation for how well a new restaurant will do in terms of brand, image and by extension how well the restaurant will perform as a business. Whether or not these assumptions are actually true in a real-world scenario is a different discussion, but this project does contain limitations in scope due to the amount of data that can be retrieved from the FourSquare API.

Nonetheless, to obtain insights into this data, we can proceed with breaking down the results of the logistic regression model. The results showed that the precision score for classifying whether the new restaurant would fall into classes 1, 2, or 3 (highest, medium, lowest) were 67%, 0%, and 78%. Therefore, the model is better at predicting if a restaurant will fall into the best or worst percentile of likes. This is good as we are mostly concerned with whether the restuarant will perform well or not so the high accuracy of predictions for the two extremum is a welcome feature. This allows us to fairly accurately predict the general performance of the business opportunity. Different binning methods for the classes were attempted, but the use of 3 bins by far yielded the best Jaccard Score.

Additionally, not only are we attempting to predict the general business performance but also pull insights to inform on business strategy. In this case strategy insight can be gleamed from the coefficient values from running the logistic regression on the full dataset. As such, we can see that opening a restaurant in Davis, or serving cuisine that is 'Latin' in nature is associated the most negatively with "likes." This suggests that the business opportunity should be opening a restaurant in either Santa Barbara or San Francisco, with a cuisine that is 'Casual' or 'American' in nature would be the best approach for maximizing likes.

# Conclusion

In conclusion, after analyzing restaurant 'likes' in California from the 280 restaurants, we can conclude that the approach to best take when looking to maximize business performance (as measured by 'likes') is to open a restaurant that is either 'Casual' or 'American' style and that opening the venue in either Santa Barbara or San Francisco rather than Davis would be the best approach. Additionally, the predictive capabilities of the logistic regression prediction model proved to be the most accurate for classifying whether a restaurant fell in either the best or worst classes when the data was binned into their 3 respective classes.